<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>（大規模言語モデルをスクラッチから構築）書籍の補足コード<br>
著者：<a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>コードリポジトリ：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# メモリ効率的なモデル重みの読み込み

- このノートブックでは、GPU（またはCPU）メモリが限られている場合に、大きな事前訓練済みまたはファインチューニング済みモデルを読み込むためのヒントを提供します
- 具体的には、`torch.save(model.state_dict(), "model.pth")`を使用してモデルを保存し（例：第5〜7章）、後で新しいセッションで継続的な事前訓練や追加のファインチューニングのために読み込みたい場合に焦点を当てています
- この例ではLLMを使用していますが、このノートブックで説明される方法は一般的であり、LLMだけでなく任意のPyTorchモデルの読み込みに適用されます

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/memory-efficient-loading/memory-efficient-loading.webp" width="800px">

In [1]:
from importlib.metadata import version

pkgs = [
    "torch",
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

torch version: 2.6.0


&nbsp;
## 1. ベンチマークユーティリティ

- まず、VRAM（GPUメモリ）を追跡するためのユーティリティコードを定義しましょう
- 後で、メインシステムRAM（CPUメモリ）を追跡するツールも導入します
- これらの関数の目的は、後で適用するときに明らかになります

In [2]:
import gc
import time
import torch


def start_memory_tracking():
    """Initialize GPU memory tracking."""
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    else:
        print("This notebook is intended for CUDA GPUs but CUDA is not available.")

def print_memory_usage():
    max_gpu_memory = torch.cuda.max_memory_allocated() / (1024 ** 3)  # Convert bytes to GB
    print(f"Maximum GPU memory allocated: {max_gpu_memory:.1f} GB")

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(3)  # some buffer time to allow memory to clear
    torch.cuda.reset_peak_memory_stats()
    max_memory_allocated = torch.cuda.max_memory_allocated(device) / (1024 ** 3)
    print(f"Maximum GPU memory allocated: {max_memory_allocated:.1f} GB")

&nbsp;
## 2. モデルのセットアップ

- このコードセクションでは、モデル自体をセットアップします
- ここでは、より興味深くするために「大きな」GPT-2モデルを使用します（このノートブックのメモリ要件と実行時間を削減するために「gpt2-small (124M)」を使用することもできます）

In [3]:
from previous_chapters import GPTModel
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel



BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-xl (1558M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

- それでは、GPUメモリ関数の動作を見てみましょう：

In [4]:
start_memory_tracking()


model = GPTModel(BASE_CONFIG)
device = torch.device("cuda")
model.to(device)

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB


- さらに、いくつかのサンプルテンソルを渡してモデルが正常に動作することを確認しましょう

In [5]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

- 次に、モデルを事前訓練して後で使用するために保存したと想像してください
- ここでは簡単にするために実際の事前訓練をスキップし、初期化されたモデルを保存するだけです（ただし、同じ概念が適用されます）

In [6]:
# Training code would go here...

model.train()
torch.save(model.state_dict(), "model.pth")

- 最後に、GPUメモリをリセットするためにPythonセッションでモデルとサンプルテンソルを削除します

In [7]:
del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 3. 重みの読み込み

- ここから、事前訓練されたモデルの重みを読み込む興味深い部分が始まります
- 以前に保存したモデルを読み込むのに必要なGPUメモリの量を見てみましょう

In [8]:
# Then load pretrained weights

start_memory_tracking()

model = GPTModel(BASE_CONFIG)
model.to(device)

model.load_state_dict(
    torch.load("model.pth", map_location=device, weights_only=True)
)
model.to(device)
model.eval();

print_memory_usage()

Maximum GPU memory allocated: 12.8 GB


- メモリが前のセッションの2倍になっていることに注意してください
- これは、短期間ですが、同じモデルをメモリ内に2回持っているためです：
  - 1回目は`model.to(device)`経由
  - 2回目は`model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))`コード行経由。最終的に、読み込まれたモデルの重みはモデルにコピーされ、`state_dict`は破棄されますが、短い間、メインモデルと読み込まれた`state_dict`の両方がメモリ内にあります
- 残りのセクションでは、この問題に対処することに焦点を当てます
- しかし、まずモデルをテストしてGPUメモリをリセットしましょう

In [9]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 4. 重みを順次読み込む

- 前のセクションで強調したように、モデルの重みをGPUメモリに2回持つ問題に対する1つの回避策は、モデルを順次読み込むことです
- 以下では：
  - 最初にモデルをGPUメモリに読み込みます
  - 次にモデルの重みをCPUメモリに読み込みます
  - 最後に各パラメータを1つずつGPUメモリにコピーします

In [10]:
start_memory_tracking()

model = GPTModel(BASE_CONFIG).to(device)

state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

print_memory_usage()

# Sequentially copy weights to the model's parameters
with torch.no_grad():
    for name, param in model.named_parameters():
        if name in state_dict:
            param.copy_(state_dict[name].to(device))
        else:
            print(f"Warning: {name} not found in state_dict.")

print_memory_usage()

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB


- 上記のように、メモリ使用量は以前よりもはるかに少ないことがわかります
- メモリが6.4から6.7 GBに増加することに注意してください。これは、最初はモデルのみがメモリにあり、次にモデルと1つのパラメータテンソルがメモリにあるためです（`".to"`を使用してモデルに割り当てることができるように、パラメータテンソルを一時的にGPUに移動します）
- 全体的に、これは大幅な改善です
- 再度、モデルを簡単にテストしてから、次のセクションのためにGPUメモリをリセットしましょう

In [11]:
# Test if the model works (no need to track memory here)
test_input = torch.tensor([[1, 2, 3]]).to(device)
model.eval()

with torch.no_grad():
    model(test_input)

del model, test_input, state_dict, param
cleanup()

Maximum GPU memory allocated: 0.0 GB


&nbsp;
## 5. 低CPUメモリでモデルを読み込む

- 前のセッションでは、重み（`state_dict`）を最初にCPUメモリに読み込んでから、モデルに1つずつコピーすることで、GPUメモリの使用を削減しました
- しかし、CPUメモリが限られている場合はどうすればよいでしょうか？
- このセクションでは、大きなGPUメモリはあるが小さなCPUメモリを持つマシンでモデルを読み込むために、PyTorchのいわゆる`"meta"`デバイスアプローチを使用します
- しかし、まずCPUメモリを監視するための便利な関数を定義しましょう

In [12]:
import os
import psutil
from threading import Thread


def memory_usage_in_gb(func, *args, **kwargs):
    process = psutil.Process(os.getpid())

    # Measure the baseline memory usage before running the function
    baseline_mem = process.memory_info().rss / 1024 ** 3  # in GB

    # Start monitoring memory in a separate thread
    mem_usage = []
    done = False

    def monitor_memory():
        while not done:
            mem_usage.append(process.memory_info().rss / 1024 ** 3)  # Convert to GB
            time.sleep(0.1)

    t = Thread(target=monitor_memory)
    t.start()

    # Run the function
    func(*args, **kwargs)

    # Stop monitoring
    done = True
    t.join()

    peak_mem_usage_gb = max(mem_usage) - baseline_mem
    return peak_mem_usage_gb


- まず、前のセクションの順次重み読み込みアプローチのCPUメモリを追跡しましょう

In [13]:
def load_sequentially():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG).to(device)

    state_dict = torch.load("model.pth", map_location="cpu", weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name].to(device))
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_sequentially)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.7 GB
-> Maximum CPU memory allocated: 6.3 GB


- 今、CPUメモリは少ないがGPUメモリは大きいマシンがあると仮定しましょう
- PyTorchのいわゆる「meta」デバイスを導入することで、CPUメモリとGPUメモリの使用をトレードオフできます
- PyTorchのmetaデバイスは、実際のメモリをデータに割り当てることなくテンソルを作成できる特別なデバイスタイプで、効果的に「meta」テンソルを作成します
- これは、メモリ割り当てのオーバーヘッドなしにテンソルの形状と型が必要なモデル分析やアーキテクチャ定義などのタスクに便利です

In [14]:
def load_sequentially_with_meta():
    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    state_dict = torch.load("model.pth", map_location=device, weights_only=True)

    print_memory_usage()

    # Sequentially copy weights to the model's parameters
    with torch.no_grad():
        for name, param in model.named_parameters():
            if name in state_dict:
                param.copy_(state_dict[name])
            else:
                print(f"Warning: {name} not found in state_dict.")

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(load_sequentially_with_meta)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 1.3 GB


- 上記のように、metaデバイスでモデルを作成し、重みを直接GPUメモリに読み込むことで、CPUメモリ要件を効果的に削減しました
- 「順次重み読み込みはまだ必要なのか、元のアプローチとどのように比較されるのか？」と疑問に思うかもしれません
- 比較のために、簡単なPyTorchの重み読み込みアプローチ（このノートブックの最初の重み読み込みセクションから）を確認しましょう：

In [15]:
def baseline():
    start_memory_tracking()

    model = GPTModel(BASE_CONFIG)
    model.to(device)

    model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
    model.to(device)
    model.eval();

    print_memory_usage()

peak_memory_used = memory_usage_in_gb(baseline)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 12.8 GB
-> Maximum CPU memory allocated: 4.4 GB


- 上記のように、metaデバイスなしの「簡単な」重み読み込みは、より多くのメモリを使用します
- つまり、CPUメモリが限られているマシンがある場合、metaデバイスアプローチを使用してモデルの重みを直接GPUメモリに読み込み、ピークCPUメモリ使用量を削減できます

&nbsp;
## 6. `mmap=True`を使用する（推奨）

- `torch.load`の中級または上級ユーザーであれば、これらのアプローチがPyTorchの`mmap=True`設定とどのように比較されるか疑問に思うかもしれません
- PyTorchの`mmap=True`設定は、メモリマップファイルI/Oを有効にし、テンソルがディスクストレージから直接データにアクセスできるようにすることで、RAMが限られている場合にファイル全体をRAMに読み込まないことでメモリ使用量を削減します
- また、[mikaylagawarecki](https://github.com/rasbt/LLMs-from-scratch/issues/402)による有用なコメントも参照してください
- 一見すると、上記の順次アプローチよりも効率が悪いように見えるかもしれません：

In [37]:
def best_practices():
  with torch.device("meta"):
      model = GPTModel(BASE_CONFIG)

  model.load_state_dict(
      torch.load("model.pth", map_location=device, weights_only=True, mmap=True),
      assign=True
  )

  print_memory_usage()

peak_memory_used = memory_usage_in_gb(best_practices)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 5.9 GB


- CPU RAM使用量が非常に高い理由は、このマシンに十分なCPU RAMが利用可能だからです
- しかし、CPU RAMが限られたマシンでこれを実行すると、`mmap`アプローチはより少ないメモリを使用します

&nbsp;
## 7. その他の方法

- このノートブックは、PyTorchで重みを読み込むためのシンプルで組み込みの方法に焦点を当てています
- CPUメモリが限られているケースでの推奨アプローチは、十分に説明した`mmap=True`アプローチです
- 別の選択肢として、各重みテンソルを個別に保存して読み込む強引なアプローチがあります：

In [13]:
model = GPTModel(BASE_CONFIG)
# Assume `model` is your trained model
state_dict = model.state_dict()

# Create a directory to store individual parameter files
os.makedirs("model_parameters", exist_ok=True)

# Save each parameter tensor separately
for name, param in state_dict.items():
    torch.save(param.cpu(), f"model_parameters/{name}.pt")

del model

In [16]:
def load_individual_weights():

    start_memory_tracking()

    with torch.device("meta"):
        model = GPTModel(BASE_CONFIG)

    model = model.to_empty(device=device)

    print_memory_usage()
    param_dir = "model_parameters"

    with torch.no_grad():
        for name, param in model.named_parameters():
            weight_path = os.path.join(param_dir, f"{name}.pt")
            if os.path.exists(weight_path):
                param_data = torch.load(weight_path, map_location="cpu", weights_only=True)
                param.copy_(param_data)
                del param_data  # Free memory
            else:
                print(f"Warning: {name} not found in {param_dir}.")

    print_memory_usage()


peak_memory_used = memory_usage_in_gb(load_individual_weights)
print(f"-> Maximum CPU memory allocated: {peak_memory_used:.1f} GB")

Maximum GPU memory allocated: 6.4 GB
Maximum GPU memory allocated: 6.4 GB
-> Maximum CPU memory allocated: 0.3 GB
